To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

In [4]:
!pip install unsloth vllm

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 633.9 kB/s eta 0:00:0000:0100:01
  Using cached http://mirrors.aliyun.com/pypi/packages/8d/cf/9b775a1a1f5fe2f6c2d321396ad41b9849de2c76fa46d78e6294ea13be91/vllm-0.7.3-cp38-abi3-manylinux1_x86_64.whl (264.6 MB)
  Using cached http://mirrors.aliyun.com/pypi/packages/fc/5f/354678fe7ead04749f8b823db7ecf830c02d504f89cc8408e80bb4e9621b/unsloth_zoo-2025.2.7-py3-none-any.whl (107 kB)
  Using cached http://mirrors.aliyun.com/pypi/packages/e5/35/0c52d708144c2deb595cd22819a609f78fdd699b95ff6f0ebcd456e3c7c1/torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl (766.6 MB)
  Using cached http://mirrors.aliyun.com/pypi/packages/2d/4a/20b2d9ac50efa0d40fbdb13283fd168cc2db28a2f21a159abbdd17a24213/xformers-0.0.29.post3-cp312-cp312-manylinux_2_28_x86_64.whl (43.4 MB)
  Using cached http://mirrors.aliyun.com/pypi/packages/20/0f/3a5f062c0ed2252ed128ff028b36d2a46a763a2919b00f12ca5274493ff3/bitsandbytes-

In [5]:
!pip install --upgrade pillow

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 7.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pillow
    Found existing installation: pillow 10.3.0
    Uninstalling pillow-10.3.0:
      Successfully uninstalled pillow-10.3.0


### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [1]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Load up `Llama 3.1 8B Instruct`, and set parameters

In [2]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/root/autodl-tmp/qwen25_R1_7b_dir/",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 03-05 00:16:12 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA L20. Max memory: 47.503 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading /root/autodl-tmp/qwen25_R1_7b_dir/ with actual GPU utilization = 59.64%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 47.5 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 256.
Unsloth: vLLM's KV Cache can use up to 14.0 GB. Also swap space = 6 GB.
INFO 03-05 00:16:48 config.py:549] This model supports multiple tasks: {'embed', 'generate', 'reward', 'score', 'classify'}. Defaulting to 'ge

[W305 00:16:49.699071206 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 03-05 00:16:53 model_runner.py:1115] Loading model weights took 14.3854 GB
INFO 03-05 00:16:53 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-05 00:16:55 worker.py:267] Memory profiling takes 1.23 seconds
INFO 03-05 00:16:55 worker.py:267] the current vLLM instance can use total_gpu_memory (47.50GiB) x gpu_memory_utilization (0.60) = 28.33GiB
INFO 03-05 00:16:55 worker.py:267] model weights take 14.39GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.41GiB; the rest of the memory reserved for KV Cache is 12.48GiB.
INFO 03-05 00:16:55 executor_base.py:111] # cuda blocks: 14605, # CPU blocks: 7021
INFO 03-05 00:16:55 executor_base.py:116] Maximum concurrency for 512 tokens per request: 456.41x
INFO 03-05 00:16:58 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory erro

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:28<00:00,  1.21it/s]

INFO 03-05 00:17:27 model_runner.py:1562] Graph capturing finished in 29 secs, took 0.35 GiB
INFO 03-05 00:17:27 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 33.36 seconds



Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


/root/autodl-tmp/qwen25_R1_7b_dir/ does not have a padding token! Will use pad_token = <|vision_pad|>.


Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [3]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('modelscope/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

Using the latest cached version of the module from /root/.cache/huggingface/modules/datasets_modules/datasets/modelscope--gsm8k/9bba91b8d001ff705b591a67e4783fd12b387d20c4ce7fd719351e07f488382e (last modified on Tue Mar  4 00:40:44 2025) since it couldn't be found locally at modelscope/gsm8k, or remotely on the Hugging Face Hub.


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [4]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [5]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 6 | Gradient Accumulation steps = 1
\        /    Total batch size = 6 | Total steps = 250
 "-____-"     Number of trainable parameters = 80,740,352


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
First, I need to determine the total number of concert tickets Mr. Benson purchased, which is 12 tickets.

Next, I calculate how many tickets exceeded the 10-ticket threshold by subtracting 10 from 12, resulting in 2 extra tickets.

Then, I apply the 5% discount to these 2 extra tickets. Calculating 5% of $40 gives a discount of $2 per extra ticket.

Now, I find the total discount by multiplying the $2 discount by the 2 extra tickets, totaling $4.

Subtracting the total discount of $4 from the original cost of 12 tickets ($480) gives the final amount Mr. Benson paid, which is $476.
</think>

Let's break down the problem step by step to find out how much Mr. Benson paid for the concert tickets.

1. **Determine the Total Number of Tickets Purchased:**
   - Mr. Benson bought *

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.000000,0.083333,0.064550,200.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000
2,-0.000000,0.020833,0.051031,200.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.041667,0.064550,200.000000,0.000192,0.041667,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.062500,0.068465,200.000000,0.000261,0.062500,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.062500,0.068465,200.000000,0.000179,0.062500,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,200.000000,0.000115,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,200.000000,0.000180,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.062500,0.068465,200.000000,0.000559,0.062500,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.083333,0.064550,200.000000,0.000231,0.083333,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.020833,0.051031,200.000000,0.000220,0.020833,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
First, calculate the total number of monthly payments for each loan by multiplying the number of years by 12.

Next, divide the total cost of each property by the number of payments to find the monthly payment amount for both the house and the trailer.

Finally, subtract the monthly payment of the trailer from the monthly payment of the house to determine how much more the house costs per month.
</think>

<reasoning>
First, calculate the total number of monthly payments for each loan:

\[
\text{Total payments per loan} = 20 \text{ years} \times 12 \text{ months/year} = 240 \text{ months}
\]

Next, determine the monthly payment for both the house and the trailer:

\[
\text{Monthly pay

TrainOutput(global_step=250, training_loss=4.9399881237480516e-05, metrics={'train_runtime': 1403.3775, 'train_samples_per_second': 1.069, 'train_steps_per_second': 0.178, 'total_flos': 0.0, 'train_loss': 4.9399881237480516e-05})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [12]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "9.11 and 9.9, which one is larger ?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
print (text)
print()
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

<｜begin▁of▁sentence｜><｜User｜>9.11 and 9.9, which one is larger ?<｜Assistant｜><think>




Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.05s/it, est. speed input: 3.47 toks/s, output: 52.21 toks/s]


"First, I need to determine which of the two numbers, 9.11 and 9.9, is larger.\n\nI'll start by comparing their whole number parts. Both numbers have a whole number part of 9, so they are equal in that aspect.\n\nNext, I'll compare their decimal parts. 9.11 has a decimal part of 0.11, while 9.9 has a decimal part of 0.9.\n\nSince 0.9 is greater than 0.11, the decimal part of 9.9 is larger than that of 9.11.\n\nTherefore, 9.9 is larger than 9.11.\n</think>\n\nTo determine which number is larger between **9.11** and **9.9**, let's compare them step by step.\n\n1. **Compare the Whole Numbers:**\n   - Both numbers have the same whole number part: **9**.\n\n2. **Compare the Decimal Parts:**\n   - **9.11** has a decimal part of **0.11**.\n   - **9.9** has a decimal part of **0.9**.\n\n3. **Determine Which Decimal Part is Larger:**\n   - **0.9** is greater than **0.11**.\n\nSince the decimal part of **9.9** is larger than that of **9.11**, the entire number **9.9** is larger.\n\n\\[\n\\boxed{

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [7]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [13]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "9.11 and 9.9, which one is larger ?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 2048,
)
print (text)
print()
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

<｜begin▁of▁sentence｜>
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
<｜User｜>9.11 and 9.9, which one is larger ?<｜Assistant｜><think>




Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.82s/it, est. speed input: 6.45 toks/s, output: 50.01 toks/s]


"I need to determine which number is larger between 9.11 and 9.9.\n\nFirst, I'll compare the integer parts of both numbers. Both have the same integer part of 9.\n\nSince the integer parts are equal, I'll compare the decimal parts.\n\nTo make the comparison easier, I'll express 9.9 as 9.90.\n\nNow, comparing 0.91 from 9.11 and 0.90 from 9.90, 0.91 is greater than 0.90.\n\nTherefore, 9.11 is larger than 9.9.\n</think>\n\n<reasoning>\nTo determine which number is larger between **9.11** and **9.9**, follow these steps:\n\n1. **Compare the Integer Parts:**\n   - Both numbers have the same integer part: **9**.\n\n2. **Compare the Decimal Parts:**\n   - Convert **9.9** to **9.90** to have the same number of decimal places as **9.11**.\n   - Now, compare the decimal parts: **0.91** (from **9.11**) and **0.90** (from **9.90**).\n\n3. **Determine Which Decimal is Larger:**\n   - **0.91** is greater than **0.90**.\n\nSince the integer parts are equal and **0.91 > 0.90**, the larger number is **

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
